# Purpose
The goal of this notebook is to make a master CSV. 



In [579]:
import pandas as pd
import re
import numpy as np
from dateutil import parser
import geocoder
from time import sleep
%matplotlib inline



## Functions

In [580]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \((.*)\)"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
def change_percent(test):
    if isinstance(test, str):
        query = r"%"
        m = re.search(query, test)

        if m:
            test = test.strip("%")
            test = float(test)
            return test
        else:
            return test
    else:
        return test
    
def get_state(test):
    query = r"([A-Z][A-Z])\Z"
    m = re.search(query, test)
    return m.group()
def get_for_dups(df):
    for_dups = []
    for index, row in df.iterrows():
        if type(row["ID"]) == str:
            for_dups.append(row["ID"])
        else:
            for_dups.append(row["Station"])
    return for_dups

def check_len(master):
    check = len(master) - 141
    if check == 0:
        print "Hurray! We didn't lose any rows."
    if check < 0:
        print "Uh oh. We lost {} rows!".format(str(abs(check)))
    if check > 0:
        print "we somehow...gained {} row? Well that's not right.".format(str(abs(check)))
        
def generate_IDs(df):
    ID = []
    for index, row in df.iterrows():
        if len(row["Station"]) > 3:
            ID.append(None)
        else:
            try:
                query = r"^\((V\d\d)\) \((\d\d\d)\) (.{2})"
                m = re.search(query, row["Location"].upper())
                new_ID = row["Station"]+m.group(3).lower()
                if new_ID not in ID:
                    ID.append(new_ID)
                else:
                    ID.append(None)
            except AttributeError:
                ID.append(None)
    return ID

def get_location_name(test):
    try:
        query = r"^\((V\d\d)\) \((.*)\) (.*)"
        m = re.search(query, test)
        return m.group(3).upper()
    except:
        return None

    
def get_qualitative_size(test):
    if test:
        if test < 1000:
            return "small"
        if test > 3000:
            return "large"
        else:
            return "average"
    else:
        return "No Size Info"
    
    
def get_geo(row):
    g = geocoder.google(row["Name"],)
    if g.county:
        return g
        #print g.confidence
    else:
        g = geocoder.google(row["Location"])
        return g
        #print g.confidence
    sleep(1.0)

### Bringing in Pending 2014 Wait Times

In [581]:
# A function to format 
def make_df(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,24,25,26],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1412 = make_df("Pending","1412","Data/Wait_Time/Pending_Clean/14_12_01_Wait_Pending.csv")

# identified these as not stations, but facilities. Dropping from dataframe. 
to_drop = [18,29,50,59,66,77,87,96,105,114,123,144,152]
for index in to_drop:
    Pending_1412 = Pending_1412.drop(index)
    
Pending_1412.head()

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,Station
0,"(V01) (402) Togus, ME",42899,2.70%,2.51,4.07,2.77,402
1,"(V01) (405) White River Junction, VT",23709,2.16%,3.92,3.95,0.79,405
2,"(V01) (518) Bedford, MA",8105,5.17%,1.12,10.53,4.41,518
3,"(V01) (523) VA Boston HCS, MA",85625,1.71%,1.66,3.53,3.92,523
4,"(V01) (608) Manchester, NH",19511,2.64%,2.66,4.21,5.13,608


# Staff Size

In [582]:
staff = pd.read_csv("Data/Staff_Size/Onboard_By_Station_By_FY.csv")
staff["Station"] = staff["ORGANIZATION"].apply(get_station)
staff["for_dups"] = get_for_dups(staff)
staff.head()
#staff

,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,MAY-FY14,MAY-FY15,MAY-FY16,ID,Station,for_dups
0,NaN,268699,272538.0,283338,294444,307430,318560,NaN,None,None
1,(V01) (402) HCS TOGUS ME,1293,1272.0,1248,1283,1335,1446,NaN,402,402
2,(V01) (405) MROC WHT RIVER JCT VT,852,853.0,881,921,1022,1177,NaN,405,405
3,(V01) (478) V1HCSBEDFORD MA,30,36.0,34,53,48,42,NaN,478,478
4,(V01) (518) MC BEDFORD MA,1313,1265.0,1245,1251,1297,1352,NaN,518,518


### Generating Unique IDs

In [583]:
Pending_1412["ID"] = generate_IDs(Pending_1412)



dup_list = Pending_1412[Pending_1412.duplicated(subset="Station",keep=False)]["ID"].tolist()
dup_list.append("612n.")
dup_list.append("537je")

for_dups = []
for index, row in Pending_1412.iterrows():
    if row["ID"] in dup_list:
        for_dups.append(row["ID"])
    else:
        for_dups.append(row["Station"])
Pending_1412["for_dups"] = for_dups



Pending_1412.head()

,Location,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,Station,ID,for_dups
0,"(V01) (402) Togus, ME",42899,2.70%,2.51,4.07,2.77,402,402to,402
1,"(V01) (405) White River Junction, VT",23709,2.16%,3.92,3.95,0.79,405,405wh,405
2,"(V01) (518) Bedford, MA",8105,5.17%,1.12,10.53,4.41,518,518be,518
3,"(V01) (523) VA Boston HCS, MA",85625,1.71%,1.66,3.53,3.92,523,523va,523
4,"(V01) (608) Manchester, NH",19511,2.64%,2.66,4.21,5.13,608,608ma,608


In [584]:
master = Pending_1412.merge(staff, on="for_dups", how="left")
master["Station"] = master["Station_x"]
master["ID"] = master["ID_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("ID_x",1).drop("ID_y",1)
print master.columns

Index([u'Location', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'for_dups', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Station', u'ID'],
      dtype='object')


#### So what was the result of our merge?

Great! Now I'm going to redefine ID so that it better matches with future dataframes.

## Bringing in Pending 2016

In [585]:
#functions to format 2016 
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,20,21,22],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Pending_1610 = make_df_2016("Pending","1610","Data/Wait_Time/Pending_Clean/16_10_01_Wait_Pending.csv")
#Pending_1610 = Pending_1610.drop(297).drop(369) # edge case causing issues, not a station anyway

In [586]:
check_len(master)
check = master

Hurray! We didn't lose any rows.


In [587]:
ID = generate_IDs(Pending_1610)
Pending_1610["ID"] = ID

In [588]:
Pending_1610 = Pending_1610.dropna(subset = ["ID"])
master = master.merge(Pending_1610,on="ID")
print master.columns
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)
master.head()

Index([u'Location_x', u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'for_dups', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16',
       u'Station_x', u'ID', u'Location_y', u'Appts_Pending_1610',
       u'%_Appts_Over_30_Pending_1610', u'PC_Wait_Pending_1610',
       u'SC_Wait_Pending_1610', u'MH_Wait_Pending_1610', u'Station_y'],
      dtype='object')


,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,for_dups,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,MAY-FY15,MAY-FY16,ID,Appts_Pending_1610,%_Appts_Over_30_Pending_1610,PC_Wait_Pending_1610,SC_Wait_Pending_1610,MH_Wait_Pending_1610,Station,Location
0,42899,2.70%,2.51,4.07,2.77,402,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,1335.0,1446.0,402to,43609,4.18%,7.82,5.67,3.62,402,"(V01) (402) Togus, ME"
1,23709,2.16%,3.92,3.95,0.79,405,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,1022.0,1177.0,405wh,21091,3.83%,6.09,7.09,1.86,405,"(V01) (405) White River Junction, VT"
2,8105,5.17%,1.12,10.53,4.41,518,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,1297.0,1352.0,518be,14456,7.54%,0.60,11.21,3.07,518,"(V01) (518) Bedford, MA"
3,85625,1.71%,1.66,3.53,3.92,523,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,4123.0,4303.0,523va,84373,4.43%,14.46,6.76,3.08,523,"(V01) (523) VA Boston HCS, MA"
4,19511,2.64%,2.66,4.21,5.13,608,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,759.0,803.0,608ma,25424,4.09%,5.75,4.58,5.21,608,"(V01) (608) Manchester, NH"


In [589]:
check_len(master)

Uh oh. We lost 1 rows!


## Bringing in Complete 2016

In [590]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \(([0-9,A-Z]{3,5})"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
        
staff["Station"] = staff["ORGANIZATION"].apply(get_station)




def make_df_2016(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,13,14,15],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1608 = make_df_2016("Complete","1608","Data/Wait_Time/Completed_Cleaned/16_08_31_Wait_Complete.csv")
Complete_1608.head()

,Location,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Wait_Complete_1608,Station
0,"(V01) (402) Togus, ME",28683,0.0134,4.96,2.73,1.87,402
1,(V01) (402) Togus VAMC,17464,0.0163,5.96,2.66,2.59,402
2,(V01) (402GA) Aroostook County\r(Caribou),756,0.0423,8.84,0.00,3.41,402GA
3,(V01) (402GB) Calais,379,0.0000,1.43,0.00,0.36,402GB
4,(V01) (402GC) Rumford,428,0.0000,1.31,0.00,1.11,402GC


In [591]:
len(Complete_1608)

1189

In [592]:
ID = generate_IDs(Complete_1608)
Complete_1608["ID"] = ID
Complete_1608 = Complete_1608.dropna(subset = ["ID"])
master = master.merge(Complete_1608,on="ID")
master["Station"] = master["Station_x"]
master["Location"] = master["Location_x"]
master= master.drop("Station_x",1).drop("Station_y",1).drop("Location_y",1).drop("Location_x",1)

In [593]:
check_len(master)

Uh oh. We lost 1 rows!


In [594]:
# Reference for duplicates I need to check out. 
master[master.duplicated(subset="Station", keep=False)][["Location","Station","ID"]]

,Location,Station,ID
8,"(V02) (528) Albany, NY",528,528al
9,"(V02) (528) Bath, NY",528,528ba
10,"(V02) (528) Western New York, NY",528,528we
11,"(V02) (528) Canandaigua, NY",528,528ca
12,"(V02) (528) Syracuse, NY",528,528sy
78,"(V15) (589) Columbia, MO",589,589co
79,"(V15) (589) Kansas City, MO",589,589ka
80,"(V15) (589) Eastern KS HCS, KS",589,589ea
81,"(V15) (589) Wichita, KS",589,589wi
82,"(V15) (657) Marion, IL",657,657ma


## Bring in Hiring

I'm going to have an issue dealing with the stations with the same ID. 

I made a new CSV called Hiring_Clean where I put the appropriate ID in a new ID column. I put ignore for those that were impossible to join accurately. 

In [595]:
Hiring = pd.read_csv("Data/Hiring/Hiring_Clean_All_Positions.csv")
Hiring["for_dups"] = get_for_dups(Hiring)
Hiring_summed = pd.pivot_table(Hiring,index=["for_dups"],values=["NbrEmps"],aggfunc=np.sum).reset_index()
Hiring_summed = Hiring_summed.rename(columns = {"NbrEmps":"Choice_Hires"})


# dup_list = master[master.duplicated(subset="Station",keep=False)]["ID"].tolist()
# dup_list.append("612n.")
# dup_list.append("537je")

# for_dups = []
# for index, row in master.iterrows():
#     if row["ID"] in dup_list:
#         for_dups.append(row["ID"])
#     else:
#         for_dups.append(row["Station"])
# master["for_dups"] = for_dups

In [596]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'for_dups', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'ID',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location'],
      dtype='object')

In [597]:
master = master.merge(Hiring_summed, on="for_dups", how="left")
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,for_dups,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,SC_Wait_Pending_1610,MH_Wait_Pending_1610,Appts_Complete_1608,%_Appts_Over_30_Complete_1608,PC_Wait_Complete_1608,SC_Wait_Complete_1608,MH_Wait_Complete_1608,Station,Location,Choice_Hires
0,42899,2.70%,2.51,4.07,2.77,402,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,5.67,3.62,28683,0.0134,4.96,2.73,1.87,402,"(V01) (402) Togus, ME",47.0
1,23709,2.16%,3.92,3.95,0.79,405,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,7.09,1.86,21581,0.0177,3.62,4.08,1.32,405,"(V01) (405) White River Junction, VT",47.0
2,8105,5.17%,1.12,10.53,4.41,518,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,11.21,3.07,13170,0.0115,0.43,6.01,0.76,518,"(V01) (518) Bedford, MA",13.0
3,85625,1.71%,1.66,3.53,3.92,523,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,6.76,3.08,46306,0.0271,7.47,5.75,2.17,523,"(V01) (523) VA Boston HCS, MA",86.0
4,19511,2.64%,2.66,4.21,5.13,608,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,4.58,5.21,18849,0.0160,4.99,2.93,2.94,608,"(V01) (608) Manchester, NH",48.0


In [598]:
check_len(master)

Uh oh. We lost 1 rows!


## Bring in Leadership

Also creating a version of the CSV to deal with duplicate stations.

In [599]:
Leadership = pd.read_csv("Data/Leadership/Leadership_cleaned.csv")
Leadership = Leadership[Leadership["ID"] != "Ignore"]
Leadership["Vacancy"] = Leadership["Acting/Detailed"] == "VACANT"
Leadership["Interim"] = Leadership["Acting/Detailed"] != "VACANT"

Leadership["for_dups"] = get_for_dups(Leadership)

Leadership_summed = pd.pivot_table(Leadership, index=["for_dups"],values=["Vacancy","Interim"],aggfunc=np.sum).reset_index()
master = master.merge(Leadership_summed, on="for_dups", how="left")

master["Interim"] = master["Interim"].fillna(0)
master["Vacancy"] = master["Vacancy"].fillna(0)











## Bringing in Completed 2014

In [600]:
pd.read_csv("Data/Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv").head()

,Location,Total Appts Completed1,Appts completed in 30 Days or under2,Percent Appts completed in 30 Days or under3,Appts completed in over 30 Days4,Percent Appts completed in over 30 Days5,Appts Completed Between 0 - 14 Days6,Appts Completed Between 15 - 30 Days7,Appts Completed Between 31 - 60 days8,Appts Completed Between 61 - 90 Days9,Appts Completed Between 91 - 120 Days10,Appts Completed Beyond 120 Days11,PC Avg Wait Time in Days12,SC Avg Wait Time in Days13,MH Avg Wait Time in Days14
0,"(V01) (402) Togus, ME",24458,24050,0.9833,408,0.0167,"23,433",617,327,57,16,8,2.72,3.34,1.43
1,"(V01) (405) White River Junction, VT",19232,18929,0.9842,303,0.0158,"18,419",510,250,40,9,4,2.24,3.22,0.84
2,"(V01) (518) Bedford, MA",11976,11786,0.9841,190,0.0159,"11,357",429,166,18,6,0,0.88,5.30,3.67
3,"(V01) (523) VA Boston HCS, MA",45669,45188,0.9895,481,0.0105,"43,900","1,288",379,77,16,9,1.16,3.17,2.63
4,"(V01) (608) Manchester, NH",17616,17329,0.9837,287,0.0163,"16,714",615,246,39,2,0,2.49,3.65,3.01


In [601]:
def make_df_complete_2014(Type,Time,File):
    Title = "{}_{}".format(Type,Time)
    csv = pd.read_csv(File,
                               usecols=[0,1,5,12,13,14],
                               skiprows=[0], #skipping the original header
                               names=["Location","Appts_{}".format(Title),
                                      "%_Appts_Over_30_{}".format(Title),
                                      "PC_Wait_{}".format(Title),
                                      "SC_Wait_{}".format(Title),"MH_Wait_{}".format(Title)])
    csv["Station"] = csv["Location"].apply(get_station)
    return csv
Complete_1409 = make_df_complete_2014("Complete","1409","Data/Wait_Time/Completed_Cleaned/14_09_30_Wait_Complete.csv")
Complete_1409["ID"] = generate_IDs(Complete_1409)
Complete_1409["for_dups"] = get_for_dups(Complete_1409)
Complete_1409

,Location,Appts_Complete_1409,%_Appts_Over_30_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409,MH_Wait_Complete_1409,Station,ID,for_dups
0,"(V01) (402) Togus, ME",24458,0.0167,2.72,3.34,1.43,402,402to,402to
1,"(V01) (405) White River Junction, VT",19232,0.0158,2.24,3.22,0.84,405,405wh,405wh
2,"(V01) (518) Bedford, MA",11976,0.0159,0.88,5.30,3.67,518,518be,518be
3,"(V01) (523) VA Boston HCS, MA",45669,0.0105,1.16,3.17,2.63,523,523va,523va
4,"(V01) (608) Manchester, NH",17616,0.0163,2.49,3.65,3.01,608,608ma,608ma
5,(V01) (631) VA Central Western Massachusetts HCS,19536,0.0500,6.31,7.80,2.91,631,631va,631va
6,"(V01) (650) Providence, RI",28521,0.0142,2.35,3.65,4.05,650,650pr,650pr
7,"(V01) (689) VA Connecticut HCS, CT",48622,0.0150,1.34,3.42,2.99,689,689va,689va
8,"(V02) (528) Albany, NY",24865,0.0088,1.28,1.86,3.02,528,528al,528al
9,"(V02) (528) Bath, NY",13553,0.0185,3.05,5.33,0.13,528,528ba,528ba


In [602]:
columns_to_use = Complete_1409.columns.difference(master.columns).tolist()
columns_to_use.append("ID")

master = master.merge(Complete_1409[columns_to_use], on="ID", how="left")
master.columns






Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'for_dups', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13', u'MAY-FY14', u'MAY-FY15', u'MAY-FY16', u'ID',
       u'Appts_Pending_1610', u'%_Appts_Over_30_Pending_1610',
       u'PC_Wait_Pending_1610', u'SC_Wait_Pending_1610',
       u'MH_Wait_Pending_1610', u'Appts_Complete_1608',
       u'%_Appts_Over_30_Complete_1608', u'PC_Wait_Complete_1608',
       u'SC_Wait_Complete_1608', u'MH_Wait_Complete_1608', u'Station',
       u'Location', u'Choice_Hires', u'Interim', u'Vacancy',
       u'%_Appts_Over_30_Complete_1409', u'Appts_Complete_1409',
       u'MH_Wait_Complete_1409', u'PC_Wait_Complete_1409',
       u'SC_Wait_Complete_1409'],
      dtype='object')

In [603]:
master.head()

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,for_dups,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,Station,Location,Choice_Hires,Interim,Vacancy,%_Appts_Over_30_Complete_1409,Appts_Complete_1409,MH_Wait_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409
0,42899,2.70%,2.51,4.07,2.77,402,(V01) (402) HCS TOGUS ME,1293.0,1272.0,1248.0,...,402,"(V01) (402) Togus, ME",47.0,0.0,1.0,0.0167,24458,1.43,2.72,3.34
1,23709,2.16%,3.92,3.95,0.79,405,(V01) (405) MROC WHT RIVER JCT VT,852.0,853.0,881.0,...,405,"(V01) (405) White River Junction, VT",47.0,0.0,0.0,0.0158,19232,0.84,2.24,3.22
2,8105,5.17%,1.12,10.53,4.41,518,(V01) (518) MC BEDFORD MA,1313.0,1265.0,1245.0,...,518,"(V01) (518) Bedford, MA",13.0,0.0,1.0,0.0159,11976,3.67,0.88,5.30
3,85625,1.71%,1.66,3.53,3.92,523,(V01) (523) HCS BOSTON MA,4022.0,4011.0,4041.0,...,523,"(V01) (523) VA Boston HCS, MA",86.0,0.0,0.0,0.0105,45669,2.63,1.16,3.17
4,19511,2.64%,2.66,4.21,5.13,608,(V01) (608) MC MANCHESTER NH,663.0,683.0,691.0,...,608,"(V01) (608) Manchester, NH",48.0,0.0,0.0,0.0163,17616,3.01,2.49,3.65


In [604]:
check_len(master)    

Uh oh. We lost 1 rows!


# Turnover

In [605]:
turnover = pd.read_csv("Data/Turnover/Turnover_Physician.csv",na_values="-")
# to drop national numbers and visn
turnover = turnover[turnover["Organization"].str.len() > 5]

turnover["Station"]= turnover["Organization"].apply(get_station)

turnover["for_dups"] = get_for_dups(turnover)

years = ["FY11","FY12","FY13","FY14","FY15"]
for year in years:
    turnover[year] = turnover[year].fillna(0)*100

turnover.head()

,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups
174,(V01) (402) HCS TOGUS ME,0602 Physician (All Specialties),9.97,9.55,11.33,14.74,7.81,NaN,402,402
175,(V01) (402) HCS TOGUS ME,01 ANESTHESIOLOGY,37.50,77.42,0.00,0.00,0.00,NaN,402,402
176,(V01) (402) HCS TOGUS ME,02 SURGERY,0.00,0.00,0.00,66.67,0.00,NaN,402,402
177,(V01) (402) HCS TOGUS ME,07 ORTHOPEDIC SURGERY,0.00,29.27,0.00,0.00,0.00,NaN,402,402
178,(V01) (402) HCS TOGUS ME,08 OTOLARYNGOLOGY,0.00,0.00,0.00,100.00,0.00,NaN,402,402


### Not merging in all of turnover

To do this would create a massive number of columns. Instead, it makes more sense to merge filtered versions in on a case by case basis.

But...it does make sense to merge in specialty 

In [606]:
turnover.to_csv("Data/turnover/Turnover_Physician_For_Merging.csv")

In [607]:

physician_turnover = pd.read_csv("Data/Turnover/Turnover_Physician_For_Merging.csv",na_values="-",
                           usecols=[2,3,4,5,6,7,10],
                           skiprows=[0], #skipping the original header
                           names=["Specialty","Physician_Turnover_FY11","Physician_Turnover_FY12",
                                  "Physician_Turnover_FY13","Physician_Turnover_FY14","Physician_Turnover_FY15",
                                  "for_dups"])
physician_turnover = physician_turnover[physician_turnover["Specialty"]=="0602 Physician (All Specialties)"].drop("Specialty",1)
master = master.merge(physician_turnover,on="for_dups",how="left")

# Looking at pyschiatrists

In [608]:
psychiatrists_hiring = pd.read_csv("Data/Hiring/Hiring_Psychiatrist.csv")
psychiatrists_hiring
psychiatrists_hiring["for_dups"] = get_for_dups(psychiatrists_hiring)
psychiatrists_hiring["NbrPsychiatrists"] = psychiatrists_hiring["NbrEmps"]

master = master.merge(psychiatrists_hiring[["NbrPsychiatrists","for_dups"]], on="for_dups", how="left")


# Fixing Percents

In [609]:
master["%_Appts_Over_30_Complete_1409"] = master["%_Appts_Over_30_Complete_1409"]*100
master["%_Appts_Over_30_Complete_1608"] = master["%_Appts_Over_30_Complete_1608"]*100
master[master["Station"]=="672"]

,Appts_Pending_1412,%_Appts_Over_30_Pending_1412,PC_Wait_Pending_1412,SC_Wait_Pending_1412,MH_Wait_Pending_1412,for_dups,ORGANIZATION,MAY-FY11,MAY-FY12,MAY-FY13,...,Appts_Complete_1409,MH_Wait_Complete_1409,PC_Wait_Complete_1409,SC_Wait_Complete_1409,Physician_Turnover_FY11,Physician_Turnover_FY12,Physician_Turnover_FY13,Physician_Turnover_FY14,Physician_Turnover_FY15,NbrPsychiatrists
51,125488,1.16%,2.42,2.61,3.33,672,(V08) (672) MC SAN JUAN PR,3346.0,3309.0,3337.0,...,77005,2.03,1.12,1.08,5.68,4.27,1.95,3.96,1.6,2.0


In [610]:
master = master.applymap(change_percent)

In [611]:
counties = pd.read_csv("Data/Vet_Pop/Vet_Pop_Counties.csv")

master["Location"]

0                                 (V01) (402) Togus, ME
1                  (V01) (405) White River Junction, VT
2                               (V01) (518) Bedford, MA
3                         (V01) (523) VA Boston HCS, MA
4                            (V01) (608) Manchester, NH
5      (V01) (631) VA Central Western Massachusetts HCS
6                            (V01) (650) Providence, RI
7                    (V01) (689) VA Connecticut HCS, CT
8                                (V02) (528) Albany, NY
9                                  (V02) (528) Bath, NY
10                     (V02) (528) Western New York, NY
11                          (V02) (528) Canandaigua, NY
12                             (V02) (528) Syracuse, NY
13                                (V03) (526) Bronx, NY
14                       (V03) (561) New Jersey HCS, NJ
15                 (V03) (620) VA Hudson Valley HCS, NY
16                  (V03) (630) New York Harbor HCS, NY
17                            (V03) (632) Northp

# Adding Vets Population

In [612]:
def get_upper(test):
    return test.upper()

# Get Name
master["Name"] = master["Location"].apply(get_location_name)
# Get State
master["State"] = master["Location"].apply(get_state)



#Bringing in VA projected veterans populations
counties = pd.read_csv("Data/Vet_Pop/Vet_Pop_Counties.csv")
counties["Projected_Increase_30_years"] = counties["9/30/43"] - counties["9/30/13"]
counties["Projected_Increase_20_years"] = counties["9/30/33"] - counties["9/30/13"]
counties["Projected_Increase_10_years"] = counties["9/30/23"] - counties["9/30/13"]
counties["Projected_Increase_5_years"] = counties["9/30/18"] - counties["9/30/13"]
#counties["State"] = counties["County, St"].apply(get_county_state)
counties["County, St"] = counties["County, St"].apply(get_upper)

#reducing number of columns
counties = counties[["Projected_Increase_5_years","Projected_Increase_10_years",
          "Projected_Increase_20_years","Projected_Increase_30_years",
                     "9/30/43","9/30/33","9/30/23","9/30/18","County, St"]]


#geocoding master
geo = []
for index, row in master.iterrows():
    add = get_geo(row)
    geo.append(add)
geo
master["Geo"] = geo

#taking geo data to get confidence and county
def get_confidence(geocode):
    return geocode.confidence
def get_county(geocode):
    county = str(geocode.county)
    #return county
    query = r"(.*) "
    m = re.search(query, county)
    try:
        return m.group(1).upper()
    except:
        return None
master["Confidence"] = master["Geo"].apply(get_confidence)
master["County"] = master["Geo"].apply(get_county)

# adding county to state for easier merging
county_st = []
for index, row in master.iterrows():
    if row["County"] == None:
        county_st.append(None) 
    else:
        add = "{}, {}".format(row["County"],row["State"])
        county_st.append(add)
master["County, St"] = county_st
master = master.merge(counties, on="County, St",how="left")


# Self Generated Columns

In [613]:
# Appts
master["Pending_Increase"] = master["Appts_Pending_1610"] - master["Appts_Pending_1412"]
master["Pending_Increase_%"] = (master["Pending_Increase"]/master["Appts_Pending_1412"])*100
master["Complete_Increase"] = master["Appts_Complete_1608"] - master["Appts_Complete_1409"]
master["Complete_Increase_%"] = (master["Complete_Increase"]/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_14"] = master["Appts_Pending_1412"] - master["Appts_Complete_1409"]
master["Complete_Pending_Appts_Diff_16"] = master["Appts_Pending_1610"] - master["Appts_Complete_1608"]

master["Complete_Pending_Appts_Diff_14_%"] = ((master["Appts_Pending_1412"] - master["Appts_Complete_1409"])/master["Appts_Complete_1409"])*100
master["Complete_Pending_Appts_Diff_16_%"] = ((master["Appts_Pending_1610"] - master["Appts_Complete_1608"])/master["Appts_Complete_1608"])*100




master["Complete_Pending_Gap_Increase_%"] = ((master["Complete_Pending_Appts_Diff_16"] -
                                             master["Complete_Pending_Appts_Diff_14"]) / master["Complete_Pending_Appts_Diff_14"])*100

# Leadership
master["Missing_Leadership"] = master["Vacancy"] + master["Interim"]

#Staff
master["Staff_11_To_12_Increase"] = master["MAY-FY12"] - master["MAY-FY11"]
master["Staff_11_To_12_Increase_%"] = (master["Staff_11_To_12_Increase"]/master["MAY-FY12"])*100

master["Staff_12_To_13_Increase"] = master["MAY-FY13"] - master["MAY-FY12"]
master["Staff_12_To_13_Increase_%"] = (master["Staff_12_To_13_Increase"]/master["MAY-FY13"])*100

master["Staff_13_To_14_Increase"] = master["MAY-FY14"] - master["MAY-FY13"]
master["Staff_13_To_14_Increase_%"] = (master["Staff_13_To_14_Increase"]/master["MAY-FY14"])*100

master["Staff_14_To_15_Increase"] = master["MAY-FY15"] - master["MAY-FY14"]
master["Staff_14_To_15_Increase_%"] = (master["Staff_14_To_15_Increase"]/master["MAY-FY15"])*100

master["Staff_15_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY15"]
master["Staff_15_To_16_Increase_%"] = (master["Staff_15_To_16_Increase"]/master["MAY-FY16"])*100

master["Staff_14_To_16_Increase"] = master["MAY-FY16"] - master["MAY-FY14"]
master["Staff_14_To_16_Increase_%"] = (master["Staff_14_To_16_Increase"]/master["MAY-FY16"])*100

# Hires
master["Choice_Increase_%"] = (master["Choice_Hires"]/master["MAY-FY14"])*100
master["Choice_Staff_Dif"] = master["Staff_14_To_16_Increase"] - master["Choice_Hires"] 
master["Choice_Increase_%_PC_Wait_Complete_1409_Ratio"] = master["PC_Wait_Complete_1409"]/master["Choice_Increase_%"]
master["Choice_Increase_%_PC_Wait_Pending_1412_Ratio"] = master["PC_Wait_Pending_1412"]/master["Choice_Increase_%"]

# Wait Times
master["PC_Wait_Pending_Increase"] = master["PC_Wait_Pending_1610"] - master["PC_Wait_Pending_1412"]
master["PC_Wait_Pending_Increase_%"] = (master["PC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["PC_Wait_Complete_Increase"] = master["PC_Wait_Complete_1608"] - master["PC_Wait_Complete_1409"]
master["PC_Wait_Complete_Increase_%"] = (master["PC_Wait_Complete_Increase"]/master["PC_Wait_Complete_1409"])*100

master["SC_Wait_Pending_Increase"] = master["SC_Wait_Pending_1610"] - master["SC_Wait_Pending_1412"]
master["SC_Wait_Pending_Increase_%"] = (master["SC_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["SC_Wait_Complete_Increase"] = master["SC_Wait_Complete_1608"] - master["SC_Wait_Complete_1409"]
master["SC_Wait_Complete_Increase_%"] = (master["SC_Wait_Complete_Increase"]/master["SC_Wait_Complete_1409"])*100

master["MH_Wait_Pending_Increase"] = master["MH_Wait_Pending_1610"] - master["MH_Wait_Pending_1412"]
master["MH_Wait_Pending_Increase_%"] = (master["MH_Wait_Pending_Increase"]/master["PC_Wait_Pending_1412"])*100

master["MH_Wait_Complete_Increase"] = (master["MH_Wait_Complete_1608"] - master["MH_Wait_Complete_1409"])
master["MH_Wait_Complete_Increase_%"] = (master["MH_Wait_Complete_Increase"]/master["MH_Wait_Complete_1409"])*100

# Staff size relative to appointments

master["Staff_To_Appts_Pending_14"] = master["Appts_Pending_1412"]/master["MAY-FY14"]
master["Staff_To_Appts_Pending_16"] = master["Appts_Pending_1610"]/master["MAY-FY16"]
master["Staff_To_Appts_Complete_14"] = master["Appts_Complete_1409"]/master["MAY-FY14"]
master["Staff_To_Appts_Complete_16"] = master["Appts_Complete_1608"]/master["MAY-FY16"]


# Appointments Over 30

master["Percentage_Point_Wait_Over_30_Increase_Pending"] = master['%_Appts_Over_30_Pending_1610'] - master['%_Appts_Over_30_Pending_1412']
master["Percentage_Point_Wait_Over_30_Increase_Complete"] = master['%_Appts_Over_30_Complete_1608'] - master['%_Appts_Over_30_Complete_1409']



# qualitative
master["qualitative_size"] = master["MAY-FY16"].apply(get_qualitative_size)


# States

In [614]:
vet_pop = pd.read_csv("Data/Vet_Pop/Vet_Pop.csv")
master = master.merge(vet_pop,on="State",how="left")


In [615]:
vet_pop

,State_Full,Vet_Pop_State_9_30_14,State,Vet_Pop_State_9_30_34,Vet_Pop_State_Increase_20
0,Alabama,413618.0,AL,359097.0,-54521.0
1,Alaska,73397.0,AK,69480.0,-3917.0
2,Arizona ...,532206.0,AZ,447300.0,-84905.0
3,Arkansas ...,249274.0,AR,210720.0,-38555.0
4,California,1851470.0,CA,1150965.0,-700505.0
5,Colorado ...,413271.0,CO,327186.0,-86086.0
6,Connecticut ...,213420.0,CT,114469.0,-98951.0
7,Delaware ...,78099.0,DE,62273.0,-15827.0
8,District of Columbia ...,29825.0,DC,23017.0,-6808.0
9,Florida ...,1583697.0,FL,1149162.0,-434534.0


# Export to CSV

In [616]:
master.to_csv("Data/Master/Master.csv", index=False)

In [617]:
master.columns

Index([u'Appts_Pending_1412', u'%_Appts_Over_30_Pending_1412',
       u'PC_Wait_Pending_1412', u'SC_Wait_Pending_1412',
       u'MH_Wait_Pending_1412', u'for_dups', u'ORGANIZATION', u'MAY-FY11',
       u'MAY-FY12', u'MAY-FY13',
       ...
       u'Staff_To_Appts_Pending_16', u'Staff_To_Appts_Complete_14',
       u'Staff_To_Appts_Complete_16',
       u'Percentage_Point_Wait_Over_30_Increase_Pending',
       u'Percentage_Point_Wait_Over_30_Increase_Complete', u'qualitative_size',
       u'State_Full', u'Vet_Pop_State_9_30_14', u'Vet_Pop_State_9_30_34',
       u'Vet_Pop_State_Increase_20'],
      dtype='object', length=103)

In [618]:
for column in master.columns:
    print column

Appts_Pending_1412
%_Appts_Over_30_Pending_1412
PC_Wait_Pending_1412
SC_Wait_Pending_1412
MH_Wait_Pending_1412
for_dups
ORGANIZATION
MAY-FY11
MAY-FY12
MAY-FY13
MAY-FY14
MAY-FY15
MAY-FY16
ID
Appts_Pending_1610
%_Appts_Over_30_Pending_1610
PC_Wait_Pending_1610
SC_Wait_Pending_1610
MH_Wait_Pending_1610
Appts_Complete_1608
%_Appts_Over_30_Complete_1608
PC_Wait_Complete_1608
SC_Wait_Complete_1608
MH_Wait_Complete_1608
Station
Location
Choice_Hires
Interim
Vacancy
%_Appts_Over_30_Complete_1409
Appts_Complete_1409
MH_Wait_Complete_1409
PC_Wait_Complete_1409
SC_Wait_Complete_1409
Physician_Turnover_FY11
Physician_Turnover_FY12
Physician_Turnover_FY13
Physician_Turnover_FY14
Physician_Turnover_FY15
NbrPsychiatrists
Name
State
Geo
Confidence
County
County, St
Projected_Increase_5_years
Projected_Increase_10_years
Projected_Increase_20_years
Projected_Increase_30_years
9/30/43
9/30/33
9/30/23
9/30/18
Pending_Increase
Pending_Increase_%
Complete_Increase
Complete_Increase_%
Complete_Pending_Appts_

# National

In [619]:
#Taken from VA site
national_complete = pd.read_csv("Data/Wait_Time/Completed_Cleaned/National_Complete.csv")
national_pending = pd.read_csv("Data/Wait_Time/Pending_Clean/National_Pending.csv")


#used to strip time from Pending_Time
def get_date(test):
    test = test.strip('National: ')
    return parser.parse(test)

national_complete["date"] = national_complete["Complete_Time"].apply(get_date)
national_pending["date"] = national_pending["Pending_Time"].apply(get_date)
national_complete.sort("date")
national_pending.sort("date")

national_pending = national_pending.drop(20) #for some reason this was read in as 1989. Just dropping it

#national_pending.to_csv("National/National_Pending.csv",index=False)
#national_complete.to_csv("National/National_Complete.csv",index=False)

#NC = national_complete[["Appts_Complete_National","date"]]
#NP = national_pending[["Appts_Pending_National","date"]]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [620]:
for column in master.columns:
    print column

Appts_Pending_1412
%_Appts_Over_30_Pending_1412
PC_Wait_Pending_1412
SC_Wait_Pending_1412
MH_Wait_Pending_1412
for_dups
ORGANIZATION
MAY-FY11
MAY-FY12
MAY-FY13
MAY-FY14
MAY-FY15
MAY-FY16
ID
Appts_Pending_1610
%_Appts_Over_30_Pending_1610
PC_Wait_Pending_1610
SC_Wait_Pending_1610
MH_Wait_Pending_1610
Appts_Complete_1608
%_Appts_Over_30_Complete_1608
PC_Wait_Complete_1608
SC_Wait_Complete_1608
MH_Wait_Complete_1608
Station
Location
Choice_Hires
Interim
Vacancy
%_Appts_Over_30_Complete_1409
Appts_Complete_1409
MH_Wait_Complete_1409
PC_Wait_Complete_1409
SC_Wait_Complete_1409
Physician_Turnover_FY11
Physician_Turnover_FY12
Physician_Turnover_FY13
Physician_Turnover_FY14
Physician_Turnover_FY15
NbrPsychiatrists
Name
State
Geo
Confidence
County
County, St
Projected_Increase_5_years
Projected_Increase_10_years
Projected_Increase_20_years
Projected_Increase_30_years
9/30/43
9/30/33
9/30/23
9/30/18
Pending_Increase
Pending_Increase_%
Complete_Increase
Complete_Increase_%
Complete_Pending_Appts_

In [636]:
locations = pd.read_csv("Data/Address/VAFacilityLocation.csv")
locations["Full_Address"] = locations["address"]+", "+locations["state"]+" "+locations["postal_code"]
locations["Station"] = locations["facility_id"]
x = master.merge(locations,on="Station",how="left")

In [680]:
y = (locations["type"]=="Intake Site (Pre-Discharge Claims Assistance)")
x = locations[(locations["Station"].str.len()==3)]
x["type"].value_counts()

Intake Site (Pre-Discharge Claims Assistance)    149
National Cemetery                                141
Community Based Outpatient Clinic                115
VA Medical Center                                107
Outpatient Clinic                                 73
Regional Benefit Office                           58
VA Health Care System                             42
VA Central Offices                                18
District                                          10
Regional Loan Center                               9
                                                   3
Community Service Programs                         2
VISN                                               2
Vet Center                                         2
Records Management Center                          1
Domiciliary                                        1
Name: type, dtype: int64

In [684]:
locations[locations["type"]=="VA Medical Center"]["Station"]

2         518
3         608
4         650
5       523A5
6         523
7       523A4
8         631
9       689A4
10        689
11        402
12        405
85      528A8
86      528A6
87      630A4
88      528A5
89      620A4
90        561
91        620
92        526
93      561A4
94        630
95        632
96      528A7
97      528A4
98        528
180       503
181       542
182       562
183       595
184       642
        ...  
853       667
854       629
855       564
931       549
932     674A4
933     671A4
934     549A4
935       740
1007      442
1008      575
1009      666
1107      531
1108      687
1109      668
1110    648A4
1111      692
1179      570
1180      640
1181      640
1182      612
1183      459
1184      640
1185      654
1186      593
1266      691
1267      605
1268      600
1269      664
1380    636A4
1381      636
Name: Station, dtype: object

In [656]:
locations["type"].value_counts()

Community Based Outpatient Clinic                755
Vet Center                                       323
Outpatient Clinic                                179
National Cemetery                                170
Intake Site (Pre-Discharge Claims Assistance)    149
VA Medical Center                                138
Regional Benefit Office                           58
VA Health Care System                             46
VA Central Offices                                21
VISN                                              20
District                                          10
                                                   9
Regional Loan Center                               9
Community Service Programs                         4
Domiciliary                                        3
Integrated Clinical Facility                       2
Records Management Center                          1
Information Resource Center                        1
Name: type, dtype: int64

In [648]:
locations[locations["Station"]=="317"]

,address,city,division,facility_id,fax,latitude,longitude,name,phone,postal_code,region,state,type,url,Full_Address,Station
1695,9500 Bay Pines Boulevard,St. Petersburg,Veterans Benefits Administration,317,NaN,27.813118,-82.771334,St. Petersburg Regional Benefit Office,800-827-1000,33708,Southeast District,FL,Regional Benefit Office,http://www.benefits.va.gov/stpetersburg/,"9500 Bay Pines Boulevard, FL 33708",317
1697,VA Joint Ambulatory Care Center,Pensacola,Veterans Benefits Administration,317,NaN,30.329175,-87.390996,Corry Station,NaN,32507,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"VA Joint Ambulatory Care Center, FL 32507",317
1698,Airman & Family Readiness Support Center,Fort Walton Beach,Veterans Benefits Administration,317,NaN,30.461526,-86.544697,Eglin Air Force Base,NaN,32542,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"Airman & Family Readiness Support Center, FL 3...",317
1706,Airman & Family Readiness Center,Hulburt Field,Veterans Benefits Administration,317,NaN,30.451584,-86.691322,Hurlburt Field,NaN,32544,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"Airman & Family Readiness Center, FL 32544",317
1707,NAS Key West & USCG Group Key West,Key West,Veterans Benefits Administration,317,NaN,24.555059,-81.779987,Key West Coast Guard,NaN,33040,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"NAS Key West & USCG Group Key West, FL 33040",317
1708,Fleet & Family Support Center Naval 1063 USS T...,Kings Bay,Veterans Benefits Administration,317,NaN,30.783237,-81.557859,Kings Bay Naval Submarine Base,NaN,31547,Southeast District,GA,Intake Site (Pre-Discharge Claims Assistance),NaN,Fleet & Family Support Center Naval 1063 USS T...,317
1709,"Base Hospital 2nd Floor, Room C 277",Tampa,Veterans Benefits Administration,317,NaN,27.844211,-82.502346,MacDill Air Force Base,NaN,33621,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"Base Hospital 2nd Floor, Room C 277, FL 33621",317
1712,Branch Medical Clinic,Mayport,Veterans Benefits Administration,317,NaN,34.720956,-77.358325,Mayport Naval Station,NaN,32228,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"Branch Medical Clinic, FL 32228",317
1713,Department of Veterans Affairs,Jacksonville,Veterans Benefits Administration,317,NaN,30.243028,-81.630908,NAS Jacksonville,NaN,32217,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"Department of Veterans Affairs, FL 32217",317
1714,NAS Key West & USCG Group Key West,Key West,Veterans Benefits Administration,317,NaN,24.555059,-81.779987,NAS Key West,NaN,33040,Southeast District,FL,Intake Site (Pre-Discharge Claims Assistance),NaN,"NAS Key West & USCG Group Key West, FL 33040",317


In [682]:
locations

,address,city,division,facility_id,fax,latitude,longitude,name,phone,postal_code,region,state,type,url,Full_Address,Station
0,150 South Huntington Avenue,Jamaica Plain,Veterans Health Administration,523,NaN,42.326988,-71.110632,VA Boston Healthcare System,617-232-9500,02130,VISN 1: VA New England Healthcare System,MA,VA Health Care System,http://www.boston.va.gov,"150 South Huntington Avenue, MA 02130",523
1,950 Campbell Avenue,West Haven,Veterans Health Administration,689,NaN,41.283665,-72.960451,VA Connecticut Healthcare System,203-932-5711,06516,VISN 1: VA New England Healthcare System,CT,VA Health Care System,NaN,"950 Campbell Avenue, CT 06516",689
2,200 Springs Rd.,Bedford,Veterans Health Administration,518,781-687-2101,42.502902,-71.272671,Edith Nourse Rogers Memorial Veterans Hospital...,781-687-2000,01730,VISN 1: VA New England Healthcare System,MA,VA Medical Center,http://www.bedford.va.gov/,"200 Springs Rd., MA 01730",518
3,718 Smyth Road,Manchester,Veterans Health Administration,608,603-626-6579,43.012300,-71.440800,Manchester VA Medical Center,603-624-4366 Or 800-892-8384,03104,VISN 1: VA New England Healthcare System,NH,VA Medical Center,http://www.manchester.va.gov,"718 Smyth Road, NH 03104",608
4,830 Chalkstone Avenue,Providence,Veterans Health Administration,650,401-457-3370,41.833799,-71.434755,Providence VA Medical Center,401-273-7100 Or 866-363-4486,02908-4799,VISN 1: VA New England Healthcare System,RI,VA Medical Center,http://www.providence.va.gov/,"830 Chalkstone Avenue, RI 02908-4799",650
5,940 Belmont Street,Brockton,Veterans Health Administration,523A5,NaN,42.065109,-71.054714,"VA Boston Healthcare System, Brockton Campus",508-583-4500 Or 508-583-4500,02301,VISN 1: VA New England Healthcare System,MA,VA Medical Center,http://www.boston.va.gov,"940 Belmont Street, MA 02301",523A5
6,150 South Huntington Avenue,Jamaica Plain,Veterans Health Administration,523,617-278-4508,42.329521,-71.110840,"VA Boston Healthcare System, Jamaica Plain Campus",617-232-9500,02130,VISN 1: VA New England Healthcare System,MA,VA Medical Center,http://www.boston.va.gov/,"150 South Huntington Avenue, MA 02130",523
7,1400 VFW Parkway,West Roxbury,Veterans Health Administration,523A4,NaN,42.270702,-71.171977,"VA Boston Healthcare System, West Roxbury Campus",617-323-7700,02132,VISN 1: VA New England Healthcare System,MA,VA Medical Center,http://www.boston.va.gov/,"1400 VFW Parkway, MA 02132",523A4
8,421 North Main Street,Leeds,Veterans Health Administration,631,413-582-3121,42.347444,-72.686019,VA Central Western Massachusetts Healthcare Sy...,413-584-4040,01053-9764,VISN 1: VA New England Healthcare System,MA,VA Medical Center,http://www.centralwesternmass.va.gov/,"421 North Main Street, MA 01053-9764",631
9,555 Willard Avenue,Newington,Veterans Health Administration,689A4,860-667-6764,41.702491,-72.734967,"VA Connecticut Healthcare System, Newington Ca...",860-666-6951,06111,VISN 1: VA New England Healthcare System,CT,VA Medical Center,http://www.connecticut.va.gov/,"555 Willard Avenue, CT 06111",689A4
